# Graph building and enrichment tools

Install libadalina-analytics using `pip`

In [1]:
pip install libadalina-analytics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Import the Amelia toolkit to read datasets.

In [2]:
from ameliadp_sql_toolkit import GrinsAmeliaSQLToolkit 
amelia_sql_toolkit = GrinsAmeliaSQLToolkit()

Read and parse OpenStreetMap dataset with roads of Milano.
Road dataset is filtered, removing minor and pedestrian only roads.

In [3]:
from libadalina_analytics.graph_extraction.readers import OpenStreetMapReader, RoadTypes

milano_roadmap_df = amelia_sql_toolkit.read_data(table_name='milano_roadmap')
osm_df = OpenStreetMapReader(RoadTypes.CAR_ONLY).from_dataframe(milano_roadmap_df)
osm_df.head()

Digit your password here:  ········


/home/jupyter-unimi.uno/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,geometry,id,name,oneway
0,"LINESTRING (9.16836 45.47604, 9.16827 45.47595...",4011790,Via Antonio Canova,forward
1,"LINESTRING (9.15073 45.45995, 9.15066 45.45993...",4011792,Via Costanza,forward
2,"LINESTRING (9.1711 45.47085, 9.17106 45.47089,...",4011793,Viale Pietro e Maria Curie,forward
3,"LINESTRING (9.15084 45.46181, 9.15076 45.46187...",4011799,Via Marchesi de' Taddei,forward
4,"LINESTRING (9.14642 45.47797, 9.14631 45.47805...",4011800,Via Monte Bianco,forward


Convert the OpenStreetMap datasets to Spark DataFrame to set the EPSG code of the geometries.
If not set explicitly, libadalina will try to infer the code from the geometries.

In [4]:
from libadalina_core.sedona_utils import to_spark_dataframe, EPSGFormats

osm_df = to_spark_dataframe(osm_df, EPSGFormats.EPSG4326)

https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/jupyter-unimi.uno/.ivy2/cache
The jars for the packages stored in: /home/jupyter-unimi.uno/.ivy2/jars
org.apache.sedona#sedona-spark-3.3_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bb8e4ecb-3dbe-4ae8-aadf-a9458ed061a9;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/jupyter-unimi.uno/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-3.3_2.12;1.7.1 in central
	found org.apache.sedona#sedona-common;1.7.1 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.error

25/08/31 14:45:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


A `networkx` `DiGraph` is built from the geometries in the OpenStreetMap dataset.

In [5]:
from libadalina_analytics.graph_extraction.builders import build_graph

graph = build_graph(osm_df, name='milan_road')
graph.number_of_nodes(), graph.number_of_edges() # number of nodes and edges

(146883, 220423)

Edge data includes information from to OpenStreetMap dataset, including the id of the geometry, and the name of the road.
Also, graph edges are enriched with the geometry of the corresponding line on the map and its distance in meters.

In [6]:
graph.edges(0, data=True) # show data of the first edge

OutEdgeDataView([(0, 223338299393, {'geometry': <LINESTRING (9.151 45.462, 9.151 45.462)>, 'id': 4011799, 'name': "Via Marchesi de' Taddei", 'distance': 13.853932347639198})])

Graphs can be further enriched with external datasets to enhance their attributes. This example demonstrates how to incorporate population data from a grid dataset, where each cell contains information about the number of residents.


In [7]:
population = amelia_sql_toolkit.read_data(table_name='milano_population')[['t', 'geometry']]
population.head()

,t,geometry
0,85,"POLYGON ((9.031086950078857 45.38280269166012,..."
1,10,"POLYGON ((9.04383459344471 45.38291898197039, ..."
2,0,"POLYGON ((9.056582284984122 45.38303373224426,..."
3,4,"POLYGON ((9.06933002405527 45.38314694247239, ..."
4,46,"POLYGON ((9.082077810016315 45.38325861264565,..."


The population data will be joined with the road network graph during the building phase to enrich each edge with population data. Specifically, it calculates the estimated population living within a 1km radius of each road segment


In [8]:
from libadalina_core.spatial_operators import AggregationFunction, AggregationType

population = to_spark_dataframe(population, EPSGFormats.EPSG4326)
graph = build_graph(osm_df,
                        name='milan_road_with_population',
                        joined_df=population,
                        buffer_radius_meters=1000, # 1km
                        aggregate_functions=[
                            AggregationFunction("t", AggregationType.SUM, 'population', proportional='geometry_right')
                        ]
                        )

25/08/31 14:46:37 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


[200.503s][warning][gc,alloc] Executor task launch worker for task 15.0 in stage 20.0 (TID 238): Retried waiting for GCLocker too often allocating 519 words


Edges data now include the amount of population nearby.

In [9]:
graph.edges(0, data=True) # show data of the first edge

OutEdgeDataView([(0, 25769803831, {'geometry': <LINESTRING (9.151 45.462, 9.151 45.462)>, 'id': 4011799, 'name': "Via Marchesi de' Taddei", 'distance': 13.853932347639198, 'population': 447.0405512516792})])